# Save courses to RDF

## Read data from Firestore

In [1]:
# init
from os import environ
environ['FIRESTORE_EMULATOR_HOST'] = 'localhost:8080'
environ['FIREBASE_AUTH_EMULATOR_HOST'] = 'localhost:8083'
environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../serviceAccountKey.json'

import firebase_admin
from firebase_admin import firestore, initialize_app, credentials

if not firebase_admin._apps:
    cred = credentials.ApplicationDefault()
    initialize_app(cred)

db = firestore.client()

In [2]:
# get a program
siit_ref = db.collection(u'programs').document(u'siit')
program = siit_ref.get().to_dict()
program['naziv']

'softversko inzenjerstvo i informacione tehnologije'

In [3]:
# get courses
courses_snapshot = siit_ref.collection(u'courses').stream()
courses = [course.to_dict() for course in courses_snapshot]

Helper Functions

In [4]:
from re import compile
def date_to_xsd_date(string_with_date):
    p = compile('(\d{2}).(\d{2}).(\d{4}).')
    day, month, year = p.search(string_with_date).groups()
    return f'{year}-{month}-{day}'

In [5]:
from rdflib import *
from Aiiso import Aiiso
from Uni import UNI
g = Graph()

g.bind('uni', UNI)
g.bind('dc', DC)
g.bind('foaf', FOAF)
g.bind('aiiso', Aiiso)

## Add program to rdf

In [6]:
program_node = URIRef(program['url'])

g.add((program_node, RDF.type, Aiiso.Programme))
g.add((program_node, UNI.name, Literal(program['naziv'])))
g.add((program_node, UNI.espb, Literal(program['espb'])))
g.add((program_node, UNI.semesters, Literal(program['brojSemestara'])))
g.add((program_node, UNI.scientificField, Literal(program['naucnoStrucneOblasti'])))
g.add((program_node, UNI.levelOfStudy, Literal(program['nivoStudija'])))
g.add((program_node, UNI.educationalField, Literal(program['obrazovnoPolje'])))
g.add((program_node, UNI.title, Literal(program['zvanje'])))

<Graph identifier=Ned76cba40b7a45c99618e179429d7994 (<class 'rdflib.graph.Graph'>)>

## Write courses to rdf

In [7]:
def addProfessors(course, course_node):
    professors = course['izvodjaciNastave']
    
    for prof in professors:
        professor_node = BNode()
        g.add((course_node, UNI.hasTeachers, professor_node))
        
        g.add((professor_node, RDF.type, FOAF.Person))
        g.add((professor_node, FOAF.name, Literal(prof['punoIme'])))
        g.add((professor_node, FOAF.title, Literal(prof['zvanje'])))
        g.add((professor_node, UNI.typeOfTeaching,
            Literal(prof['vidNastave'])))


In [8]:
def addLiterature(course, course_node):
    literature = course['literatura']

    for lit in literature:
        book_node = BNode()

        g.add((course_node, UNI.hasLiterature, book_node))
        g.add((book_node, RDF.type, UNI.Book))
        if lit['naziv']: g.add((book_node, DC.title, Literal(lit['naziv'])))
        if lit['autori']: g.add((book_node, DC.creator, Literal(lit['autori'])))
        if lit['godina']: g.add((book_node, DC.date, Literal(lit['godina'])))
        if lit['izdavac']: g.add((book_node, DC.publisher, Literal(lit['izdavac'])))
        if lit['jezik']: g.add((book_node, DC.language, Literal(lit['jezik'])))


In [9]:
def addEvaluation(course, course_node):
    evaluation = course['formiranjeOcene']
    
    for e in evaluation:
        evaluation_node = BNode()

        g.add((course_node, UNI.hasEvaluation, evaluation_node))
        g.add((evaluation_node, RDF.type, UNI.Evaluation))
        g.add((evaluation_node, UNI.points, Literal(e['brojPoena'])))
        g.add((evaluation_node, UNI.required, Literal(e['obavezna'])))
        g.add((evaluation_node, UNI.preExamination, Literal(e['predispitna'])))
        g.add((evaluation_node, UNI.activity, Literal(e['predmetnaAktivnost'])))

In [10]:
for i, course in enumerate(courses):
    course_node = URIRef(course['url'])

    g.add((course_node, RDF.type, Aiiso.Course))
    g.add((course_node, DC.title, Literal(course['naziv'])))
    g.add((course_node, DC.description, Literal(course['sadrzaj'])))
    g.add((course_node,DC.date, Literal(date_to_xsd_date(course['osnovneInformacije']), datatype=XSD.date)))
    g.add((course_node, DC.type, Literal(course['kategorija'])))
    g.add((course_node, UNI.scientificField, Literal(course['naucnaOblast'])))

    g.add((course_node, UNI.purpose, Literal(course['cilj'])))
    g.add((course_node, UNI.espb, Literal(course['espb'])))
    g.add((course_node, UNI.year, Literal(course['godina'])))
    g.add((course_node, UNI.result, Literal(course['ishod'])))
    g.add((course_node, UNI.methodology, Literal(course['metodologijaIzvodjenjaNastave'])))
    g.add((course_node, UNI.semester, Literal(course['semestar'])))
    
    addProfessors(course, course_node)
    addLiterature(course, course_node)
    addEvaluation(course, course_node)


## Write tests to rdf

In [12]:
def addQuestionsAndAnswers(test, test_node, testuri):
    questions_snapshot = db.collection(f'tests/{test["id"]}/questions').stream()

    for q in questions_snapshot:
        #create question node
        question = q.to_dict()
        question['id'] = q.id
        # question_uri = f'{testuri}/questions/{question["id"]}'
        question_node = BNode()
        g.add((question_node, RDF.type, UNI.Question))
        g.add((question_node, UNI.text, Literal(question['text'])))
        g.add((question_node, UNI.points, Literal(question['maxPoints'])))
        g.add((question_node, UNI.domainProblem, Literal(question['domainProblemName'])))
        
        # add question node to the test node
        g.add((test_node, UNI.hasQuestions, question_node))

        # add possible answers to the question node
        [g.add((question_node, UNI.possibleAnswers, Literal(ans))) for ans in question['possibleAnswers']]
        
        # add correct answers to the question node
        correct_answers = db.document(f'tests/{test["id"]}/answers/{question["id"]}').get().to_dict()['correctAnswers']
        [g.add((question_node, UNI.correctAnswers, Literal(ans))) for ans in correct_answers]

In [13]:
# get tests
tests_snapshot = db.collection('tests').stream()

for test_snap in tests_snapshot:
    test = test_snap.to_dict()
    test['id'] = test_snap.id
    testuri = "http://www.sotis/tests/" + test['id']
    test_node = URIRef(testuri)

    g.add((test_node, RDF.type, UNI.Test))
    g.add((test_node, UNI.name, Literal(test['name'])))
    g.add((test_node, UNI.points, Literal(test['maxPoints'])))

    # connect test & course. Test has property courseName, by which we can find a course
    course_url = next((s for s, p, o in g.triples((None, DC.title, Literal(test['courseName'])))), None)
    if(course_url): g.add((test_node, UNI.partOf, course_url))
    addQuestionsAndAnswers(test, test_node, testuri)

## Write takes to rdf

In [14]:
def addMyAnswers(user, take, take_node):
    answers_snapshot = db.collection(f'users/{user["uid"]}/takes/{take["id"]}/myAnswers').stream()
    
    for ans_snap in answers_snapshot:
        answer = ans_snap.to_dict()
        answer_node = BNode()
        g.add((take_node, UNI.hasAnswers, answer_node))
        g.add((answer_node, RDF.type, UNI.MyAnswer))
        g.add((answer_node, UNI.points, Literal(answer['points'])))
        g.add((answer_node, UNI.correct, Literal(answer['correct'])))
        [g.add((answer_node, UNI.answersList, Literal(ans))) for ans in answer['myAnswers']]
        

In [15]:
def addTakes(user):
    takes_snapshot = db.collection(f'users/{user["uid"]}/takes').stream()
    for take_snap in takes_snapshot:
        take = take_snap.to_dict()
        take['id'] = take_snap.id
        take_node = URIRef(f'http://www.sotis/takes/{take["id"]}')

        # add take to the graph
        g.add((take_node, RDF.type, UNI.Take))
        g.add((take_node, UNI.startTime, Literal(take['startTime'])))
        g.add((take_node, UNI.endTime, Literal(take['endTime'])))
        g.add((take_node, UNI.points, Literal(take['points'])))
        g.add((take_node, UNI.passed, Literal(take['passed'])))
        
        test_node = URIRef(f'http://www.sotis/tests/{take["testId"]}')
        g.add((take_node, UNI.partOf, test_node))

        user_node = URIRef(f'http://www.sotis/users/{u.id}')
        g.add((user_node, RDF.type, FOAF.Person))
        g.add((user_node, FOAF.name, Literal(user['displayName'])))    
        g.add((take_node, UNI.doneBy, user_node))
    
        addMyAnswers(user, take, take_node)

In [16]:
users_snapshot = db.collection(u'users').stream()
for u in users_snapshot:
    user = u.to_dict()
    if(user['role'] == "student"):
        addTakes(user)


In [19]:
# save rdf data into file
g.serialize('uni.ttl')

<Graph identifier=Ned76cba40b7a45c99618e179429d7994 (<class 'rdflib.graph.Graph'>)>